# Ollama를 활용한 LLM prompt engineering 기초

- 작성자 임이로 bkksg.studio@gmail.com
- python 3.12.3사용중

In [31]:
!nvidia-smi #NVIDIA GPU 상태를 보여주는 표준 유틸리티. # MIG(Multi-Instance GPU) : Disabled
!python -c "import torch,platform,sys; print('py',sys.version); print('cuda?',torch.cuda.is_available())" #True면 PyTorch가 GPU(CUDA) 버전을 잘 찾았다는 뜻.

Mon Sep 15 14:26:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:49:00.0 Off |                    0 |
| N/A   37C    P0             36W /  250W |   13041MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

#### ollama를 클라우드 서버에 설치

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh #https://ollama.com/ 

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%                                                            5.8%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#### ollama serve 시작

In [8]:
import subprocess, os
# subprocess : 외부 명령어나 프로그램을 파이썬 코드에서 실행할 수 있게 해주는 모듈.
# os : 운영체제 관련 작업을 할 때 쓰는 모듈. / 환경변수 설정, 디렉토리 만들기/바꾸기, 경로 조작 등에 활용.

In [40]:
# # GPU 환경변수 설정 후 서버 실행
# !export OLLAMA_USE_GPU=1
# # 선택: 특정 GPU만 쓰려면 (예: 0번 GPU만)
# !export CUDA_VISIBLE_DEVICES=0

os.environ["OLLAMA_USE_GPU"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 필요 시 선택

In [43]:
# 주피터 프로세스 자체의 환경변수 수정
os.environ["OLLAMA_MODELS"] = "../../ollama"

# serve 띄울 때도 같은 os.environ 사용
subprocess.Popen(["ollama", "serve"], env=os.environ)


<Popen: returncode: None args: ['ollama', 'serve']>

time=2025-09-15T14:58:57.998+09:00 level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:../../ollama OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:

In [44]:
!ollama list

[GIN] 2025/09/15 - 14:59:00 | 200 |      60.625µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/15 - 14:59:00 | 200 |   35.640505ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED       
gemma3:27b     a418f5838eaf    17 GB    21 minutes ago    
gpt-oss:20b    aa4295ac10c3    13 GB    8 days ago        


#### ollama server 종료

In [41]:

subprocess.run(["pkill", "-f", "ollama serve"], check=False)

CompletedProcess(args=['pkill', '-f', 'ollama serve'], returncode=0)

#### ollama pull llm

In [12]:
# 모델 풀
subprocess.run(["ollama", "pull", "gpt-oss:20b"], check=True, env=os.environ)

[GIN] 2025/09/15 - 13:45:00 | 200 |      79.129µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ time=2025-09-15T13:45:01.302+09:00 level=INFO source=download.go:177 msg="downloading b112e727c6f1 in 16 861 MB part(s)"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 729 KB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 1.4 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 2.8 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏ 5.9 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   0% ▕                  ▏  24 MB/ 13 GB                  pulling manifest 
pulling b112e727c6f1:   1% ▕                  ▏ 146 MB/ 13 GB           

[GIN] 2025/09/15 - 13:47:43 | 200 |         2m43s |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling b112e727c6f1: 100% ▕██████████████████▏  13 GB                         
pulling fa6710a93d78: 100% ▕██████████████████▏ 7.2 KB                         
pulling f60356777647: 100% ▕██████████████████▏  11 KB                         
pulling d8ba2f9a17b3: 100% ▕██████████████████▏   18 B                         
pulling 55c108d8e936: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gpt-oss:20b'], returncode=0)

In [14]:
# 모델 풀
subprocess.run(["ollama", "pull", "gemma3:27b"], check=True, env=os.environ)

[GIN] 2025/09/15 - 13:50:02 | 200 |      29.159µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ time=2025-09-15T13:50:03.630+09:00 level=INFO source=download.go:177 msg="downloading e796792eba26 in 18 1 GB part(s)"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 398 KB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 770 KB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 1.9 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 3.2 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏ 4.6 MB/ 17 GB                  pulling manifest 
pulling e796792eba26:   0% ▕                  ▏  69 MB/ 17 GB                  pulling manife

[GIN] 2025/09/15 - 13:52:50 | 200 |         2m48s |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest ⠸ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest ⠼ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a425

CompletedProcess(args=['ollama', 'pull', 'gemma3:27b'], returncode=0)

### ollama - gpt-oss:20b

In [38]:
!pip install -q langchain langchain-ollama
!pip install -q langchain-community

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1

In [45]:
import requests
resp = requests.post(
    "http://127.0.0.1:11434/api/generate",
    json={"model": "gpt-oss:20b", "prompt": "GPU로 추론 중인지 확인하는 간단한 방법을 한 문장으로 말해줘(한국어).", "stream": False},
    timeout=120,
)
print(resp.json()["response"])


time=2025-09-15T14:59:26.191+09:00 level=INFO source=server.go:199 msg="model wants flash attention"
time=2025-09-15T14:59:26.192+09:00 level=INFO source=server.go:216 msg="enabling flash attention"
time=2025-09-15T14:59:26.192+09:00 level=WARN source=server.go:224 msg="kv cache type not supported by model" type=""
time=2025-09-15T14:59:26.192+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model ../../ollama/blobs/sha256-b112e727c6f18875636c56a779790a590d705aec9e1c0eb5a97d51fc2a778583 --port 45965"
time=2025-09-15T14:59:26.221+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-15T14:59:26.221+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:45965"
time=2025-09-15T14:59:26.502+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="847.2 GiB" free_swap="8.0 GiB"
time=2025-09-15T14:59:26.779+09:00 level=INFO source=memory.go:36 msg="new model 

[GIN] 2025/09/15 - 14:59:35 | 200 | 11.565594754s |       127.0.0.1 | POST     "/api/generate"
Python에서 PyTorch를 사용한다면 `torch.cuda.is_available()`가 `True`라면 현재 GPU를 활용해 추론 중인 것입니다.


In [28]:
!ollama list

[GIN] 2025/09/15 - 14:24:49 | 200 |      33.159µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/15 - 14:24:49 | 200 |    2.118495ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED   
gemma3:27b     a418f5838eaf    17 GB    8 days ago    
gpt-oss:20b    aa4295ac10c3    13 GB    8 days ago    


In [48]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

# Ollama 연결
llm = ChatOllama(
    base_url="http://localhost:11434",  # ollama serve가 켜져 있는 주소
    model="gpt-oss:20b",
    temperature=0.1,
    num_ctx=2048,
)

# from langchain_community.llms import Ollama
# from langchain_core.messages import SystemMessage, HumanMessage

# llm = Ollama(
#     base_url="http://127.0.0.1:11434",  # ollama serve 실행 중인 주소
#     model="gpt-oss:20b",
#     temperature=0.3,
#     num_ctx=2048,
# )

In [49]:
# 단일 메시지
resp = llm.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)


[GIN] 2025/09/15 - 15:02:26 | 200 |         1m10s |       127.0.0.1 | POST     "/api/chat"
## 장곡사(장곡사)와 미륵불·괘불탱에 대한 개요

| 항목 | 내용 |
|------|------|
| **위치** | 강원도 평창군 장곡면 장곡리 (평창군 중앙에 위치한 산속 절) |
| **설립 연도** | 1395년(조선 태조 이성계 시기) |
| **주요 사찰 건물** | 미륵불(미륵불탱), 괘불탱, 정전(정전), 사찰 복합 건물 등 |
| **대표 유물** | 12m·1,200톤 규모의 동형 미륵불, 괘불탱(미륵불을 수용하는 정교한 목조 탱) |

---

### 1. 장곡사의 역사와 배경

- **설립 배경**  
  장곡사는 조선 초기 왕조의 건국과 함께 설립된 사찰로, 평창 지역의 산림과 물이 풍부한 환경에 자리 잡고 있습니다. 초기에는 ‘장곡사’라는 이름으로 불렸으며, 조선시대에는 ‘장곡사’라는 이름이 그대로 유지되었습니다.

- **역사적 변천**  
  - **1395년**: 조선 태조 이성계가 설립.  
  - **조선 후기**: 사찰이 여러 차례 화재와 전쟁으로 피해를 입었으나, 지역 주민과 왕실의 후원으로 재건되었습니다.  
  - **현대**: 1999년 대형 미륵불이 세워지면서 사찰의 명성이 크게 상승했습니다. 이후 2000년대 초반에는 괘불탱이 건립되어 미륵불을 보호하고 전시하는 역할을 하게 되었습니다.

---

### 2. 미륵불(미륵불탱)

| 항목 | 내용 |
|------|------|
| **형태** | 동형(동물 형태) 미륵불, 12m 높이, 1,200톤 무게 |
| **재료** | 주로 청동(동)과 금속 합금 |
| **제작 시기** | 1999년 (현대에 제작) |
| **의의** | 미륵보살(미래의 부처)을 상징하며, 한국 사찰 중 가장 큰 미륵불 중 하나입니다. |
| **관람 포인트** | - 미륵불 내부에 작은 사원(미륵보살의 상징적 공간) <b

In [51]:
# 시스템 message / propmt (역할지정 / 명령어 지정) -> 이부분을 작성을 잘해야함. https://wikidocs.net/285650 (위키독스 진행)

# llm = ChatOllama(base_url="http://127.0.0.1:11434", model="gpt-oss:20b", temperature=0.3)
resp = llm.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 15:05:08 | 200 |         1m33s |       127.0.0.1 | POST     "/api/chat"
## 장곡사 미륵불과 괘불탱(괘불탱)

| 항목 | 내용 |
|------|------|
| **위치** | 경상북도 경주 시내 장곡사(장곡사 미륵불전) |
| **시대** | 신라 7세기(약 7세기 중반) |
| **문화재 등급** | 국보 제1호(미륵불) / 국보 제2호(괘불탱) |
| **주요 특징** | 1. **미륵불** – 2.5m 높이의 청동 조각, 신라 금속공예의 정수<br>2. **괘불탱** – 미륵불을 수용하는 작은 석조 신전 구조(‘괘불탱’이라 부름) |

---

### 1. 장곡사 미륵불 (국보 제1호)

| 구분 | 세부 내용 |
|------|-----------|
| **형태** | 좌상(좌우로 앉아 있는 형태) |
| **재료** | 청동(주조) |
| **크기** | 높이 약 2.5m, 무게 약 1.5톤 |
| **특징** | - **표정**: 부드러운 미소와 눈빛으로 ‘미래의 평화’를 상징<br>- **손짓**: 오른손은 ‘아바야(무공포)’ mudra, 왼손은 작은 그릇을 들고 있음<br>- **의복·머리**: 복잡한 실루엣과 섬세한 장식(의복의 주름, 머리카락의 흐름)까지 정교하게 표현<br>- **공예적 가치**: 신라 청동 조각의 최고 수준으로, 금속의 부드러운 질감과 세밀한 디테일이 인상적 |

> **의미**  
> 미륵불은 ‘미래의 부처’로서, 인류가 겪는 고통을 넘어서는 평화와 구원을 약속합니다. 장곡사의 미륵불은 그 상징성을 물리적 형태로 구현한 대표작으로, 신라의 종교적·문화적 비전을 한눈에 보여 줍니다.

---

### 2. 괘불탱 (국보 제2호)

| 구분 | 세부 내용 |
|------|-----------|
| **형태** | 석조 ‘괘불탱’(Gwaebul‑tang) – 작은 탑형 신전 구조<br> 1층은 석조 벽, 2층은 지붕(삼각형 또는 

### ollama - gemma3:27b

In [ ]:
# import os, subprocess

# # 원하는 경로 지정 (여기선 ./models)
# os.environ["OLLAMA_MODELS"] = "../../ollama"

# # 디렉토리가 없으면 먼저 생성
# os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# # gemma3:12b 모델 풀
# subprocess.run(["ollama", "pull", "gemma3:27b"], check=True, env=os.environ)


[GIN] 2025/09/15 - 14:37:51 | 200 |      32.518µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/15 - 14:37:52 | 200 |   728.64908ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest ⠦ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gemma3:27b'], returncode=0)

In [50]:
# Ollama 연결
llm1 = ChatOllama(
    base_url="http://127.0.0.1:11434",  # ollama serve가 켜져 있는 주소
    model="gemma3:27b",
    temperature=0.1,
    num_ctx=2048,
)

In [36]:
# 단일 메시지
resp = llm1.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)

time=2025-09-15T14:29:52.985+09:00 level=INFO source=sched.go:540 msg="updated VRAM based on existing loaded models" gpu=GPU-9fc64c71-9800-370c-ee71-f6a0a869bc7a library=cuda total="39.5 GiB" available="26.3 GiB"
time=2025-09-15T14:29:53.580+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model ../../ollama/blobs/sha256-e796792eba26c4d3b04b0ac5adb01a453dd9ec2dfd83b6c59cbf6fe5f30b0f68 --port 43267"
time=2025-09-15T14:29:53.609+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-15T14:29:53.609+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:43267"
time=2025-09-15T14:29:54.496+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="843.7 GiB" free_swap="8.0 GiB"
time=2025-09-15T14:29:54.499+09:00 level=INFO source=memory.go:36 msg="new model will fit in available VRAM across minimum required GPUs, loading" model=../../ollama/blobs/sha256-e79679

[GIN] 2025/09/15 - 14:31:15 | 200 |         1m23s |       127.0.0.1 | POST     "/api/chat"
장곡사 미륵불 괘불탱은 경상남도 거창군 북상면 장곡사에 있는 괘불탱으로, 조선 후기에 제작된 불교 회화의 걸작입니다. 괘불탱은 불교 의식 때 법당에 걸어놓는 대형 불화로, 장곡사 괘불탱은 특히 뛰어난 예술성과 역사적 가치를 인정받아 국보 제135호로 지정되었습니다.

**주요 특징:**

*   **미륵보살의 광대한 세계:** 괘불탱의 중심에는 미래에 인간으로 다시 태어나 중생을 구제할 미륵보살이 웅장하게 자리하고 있습니다. 미륵보살은 화려한 보관을 쓰고, 다양한 장신구를 착용하고 있으며, 손에는 중생을 구제하는 상징인 여원인을 들고 있습니다. 미륵보살 주변에는 보살, 천인, 동자 등 다양한 부속 존재들이 묘사되어 있어 미륵보살의 광대한 세계를 표현하고 있습니다.
*   **화려하고 섬세한 표현:** 괘불탱은 전체적으로 화려하고 섬세한 표현이 돋보입니다. 채색 기법이 뛰어나고, 인물들의 표정과 옷 주름, 장신구 등이 정교하게 묘사되어 있습니다. 특히, 미륵보살의 얼굴은 자비롭고 온화한 인상을 주며, 보는 이로 하여금 경건한 마음을 갖게 합니다.
*   **조선 후기 불화의 특징:** 장곡사 괘불탱은 조선 후기 불화의 특징을 잘 보여줍니다. 이전 시대에 비해 인물 표현이 사실적으로 변화하고, 배경 묘사가 간략해지는 경향을 보입니다. 또한, 민화의 영향으로 인해 그림에 생동감이 더해졌습니다.
*   **제작 배경:** 장곡사 괘불탱은 18세기 후반에 제작된 것으로 추정됩니다. 정확한 제작 시기와 화가는 알려져 있지 않지만, 당시 불교계의 상황과 불화의 특징을 고려할 때, 조선 후기 대표적인 불화 화가 중 한 명이 제작했을 것으로 추정됩니다.

**장곡사 괘불탱의 가치:**

*   **예술적 가치:** 장곡사 괘불탱은 뛰어난 예술성을 바탕으로 불교 회화의 정수를 보여줍니다. 화려하고 섬세한 표현, 균형 

In [37]:
resp = llm1.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 14:32:10 | 200 | 22.733723709s |       127.0.0.1 | POST     "/api/chat"
안녕하세요! AI 문화유산 스마트 도슨트입니다. 장곡사 미륵불 괘불탱에 대해 자세히 설명해 드리겠습니다.

**장곡사 미륵불 괘불탱 (長谷寺彌勒佛掛佛幀)**

*   **지정:** 보물 제1357호
*   **소장:** 전라북도 정읍시 장곡사
*   **제작시기:** 조선 후기 (추정)
*   **크기:** 가로 6.8m, 세로 10.3m

**괘불탱이란?**

괘불탱은 절에서 중요한 의식을 치를 때 법당에 걸어놓는 대형 불교 회화입니다. 괘불탱은 단순히 그림이 아니라, 의식의 중요한 도구이자 신앙의 대상입니다. 특히 장곡사 괘불탱은 현존하는 괘불탱 중 가장 큰 규모를 자랑하며, 뛰어난 예술적 가치를 인정받고 있습니다.

**장곡사 괘불탱의 특징**

*   **미륵보살의 광대한 세계:** 괘불탱의 중앙에는 미래에 인간으로 다시 태어나 중생을 구제할 미륵보살이 웅장하게 자리하고 있습니다. 미륵보살은 화려한 보관을 쓰고, 온화한 미소를 짓고 있으며, 손에는 중생을 구제하는 상징인 여의주를 들고 있습니다. 미륵보살 주변에는 다양한 보살과 천인, 그리고 극락 세계를 상징하는 연꽃과 구름이 묘사되어 있어, 광대한 미륵보살의 세계를 표현하고 있습니다.
*   **화려하고 섬세한 채색:** 괘불탱은 밝고 화려한 오채를 사용하여 그려졌습니다. 특히, 미륵보살의 옷과 보관, 그리고 주변 보살들의 장신구는 금박을 사용하여 빛을 발하며, 섬세하고 정교한 묘사로 보는 이의 감탄을 자아냅니다.
*   **조선 후기 불화의 정수:** 장곡사 괘불탱은 조선 후기 불화의 특징을 잘 보여줍니다. 화려하고 장식적인 표현, 그리고 섬세하고 정교한 묘사는 당시 불화의 높은 수준을 보여줍니다. 또한, 괘불탱에는 당시 사회의 문화와 사상을 반영하는 다양한 상징과 문양이 담겨 있어, 역사적, 문화적 가치가 높습니다.
*   **희귀한 구도